# MAUDE 데이터 전처리

## 라이브러리 불러오기

In [1]:
from typing import Tuple, List, Union, Iterator, Dict
import polars as pl
from pprint import pprint
import plotly.graph_objects as go
import sys
from pathlib import Path
import re

# 상대 경로 사용
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'

# 맨 앞에 추가
sys.path.insert(0, str(PROJECT_ROOT))

# Python 내장 code 모듈 캐시만 임시 제거
if 'code' in sys.modules:
    del sys.modules['code']

# 이제 import
from code.loading import DataLoader

In [2]:
import platform
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm

# 한글 폰트 설정
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
elif platform.system() == 'Darwin':  # macOS
    plt.rcParams['font.family'] = 'AppleGothic'
else:  # Linux
    plt.rcParams['font.family'] = 'NanumGothic'

plt.rcParams['axes.unicode_minus'] = False

mpl.rcParams['axes.unicode_minus'] = False
# 폰트 개인 경로에 맞춰서 변경
# FONT_DIR = Path("/path/to/fonts")
# font_path = FONT_DIR / 'FREESENTATION-6SEMIBOLD.ttf'
# prop = fm.FontProperties(fname=font_path)

## 데이터 불러오기

In [3]:

loader2 = DataLoader(
    start=2020,
    end=2025,
    output_file=DATA_DIR / 'maude.parquet',
    max_workers=4
)

adapter = 'polars'
polars_kwargs = {
    'use_statistics': True,
    'parallel': 'auto',
    'low_memory': False,
    'rechunk': False,
    'cache': True,
}
lf = loader2.load(adapter=adapter, **polars_kwargs)
lf


📖 /home/dataiku/eric/Sparta/Project3/data/maude.parquet 로딩 중... (adapter=polars)


### 사용할 컬럼

In [4]:
# 기본 변수
BASE_COLS = [
    'mdr_report_key', 'adverse_event_flag', 'product_problem_flag', 'date_of_event',
    'date_received', 'device_date_of_manufacturer', 'event_type',
    'previous_use_code', 'single_use_flag', 'report_source_code',
    'reprocessed_and_reused_flag', 'report_to_fda', 'event_location', 'manufacturer_link_flag', 'pma_pmn_number'
]

DEVICE_PATTERNS = [
    r"^device_.*_brand_name$",
    r"^device_.*_udi_di$",
    r"^device_.*_device_report_product_code$",
    r"^device_.*_model_number$",
    r"^device_.*_device_age_text$",
    r"^device_.*_device_operator$",
    r"^device_.*_manufacturer_d_name$",
    r"^device_.*_openfda_device_class$",
    r"^device_.*_openfda_device_name$",
]


PATIENT_PATTERNS = [
    r"^patient_.*_patient_sequence_number$",
    r"^patient_.*_patient_age$",
    r'^patient_.*_patient_sex$',
    r"^patient_.*_patient_weight$",
    r"^patient_.*_patient_race$",
    r"^patient_.*_patient_problems$",
    r"^patient_.*_sequence_number_outcome$",
    r"^patient_.*.sequence_number_treatment$",
]


MDR_TEXT_PATTERNS = [
    r"^mdr_text_.*_text$",
    r"^mdr_text_.*_text_type_code$",
]

In [5]:
my_cols = [
    'report_to_fda',
    'event_location',
    'mdr_report_key',
    'manufacturer_link_flag',
    'device_0_openfda_device_class',
    'device_0_openfda_device_name',
    'device_0_brand_name',
]

## 공용 함수

In [6]:
import re
from typing import List

def get_pattern_cols(
    lf: pl.LazyFrame,
    pattern: List[str],
) -> List[str]:
    
    cols = lf.collect_schema().names()
    regexes = [re.compile(p) for p in pattern]
    
    return [c for c in cols if any(r.search(c) for r in regexes)]

In [7]:
def get_use_cols(
    lf: pl.LazyFrame,
    patterns: Dict[str, List[str]],
    base_cols: List[str] = BASE_COLS,
) -> Tuple[List[str], List[str], List[str], List[str]]:
    
    analysis_cols = base_cols
    
    # 합치기
    pattern_cols = {}
    for k, pattern in patterns.items():
        pattern_cols[k] = get_pattern_cols(lf, pattern)
        analysis_cols += pattern_cols[k]

    analysis_cols = sorted(list(set(analysis_cols)), reverse=True)

    print(f"총 컬럼: {len(analysis_cols)}개")
    for k, pattern in pattern_cols.items():
        print(f"{k} 컬럼: {len(pattern)}개")

    return analysis_cols, pattern_cols

patterns = {
    'device': DEVICE_PATTERNS,
    'patient': PATIENT_PATTERNS,
    'mdr': MDR_TEXT_PATTERNS
}
analysis_cols, pattern_cols = get_use_cols(lf, patterns)

총 컬럼: 700개
device 컬럼: 495개
patient 컬럼: 30개
mdr 컬럼: 160개


In [8]:
def draw_donut_chart(count_df: pl.DataFrame, col: str, top_n: int = 5) -> None:
    # top_n이 지정되면 처리
    if top_n and len(count_df) > top_n:
        top = count_df.head(top_n)
        rest_sum = count_df[top_n:].select(pl.col('count').sum()).item()
        
        if rest_sum > 0:
            rest_percentage = round(rest_sum / count_df.select(pl.col('count').sum()).item() * 100, 2)
            other_row = pl.DataFrame({
                col: ['Minor Group'],
                'count': [rest_sum],
                'percentage': [rest_percentage]
            }).with_columns(
                pl.col('count').cast(count_df['count'].dtype)
            )
            count_df = pl.concat([top, other_row])
        else:
            count_df = top
    
    # 도넛 차트 생성
    fig = go.Figure(data=[go.Pie(
        labels=count_df[col],
        values=count_df['count'],
        hole=.4,
        hoverinfo="label+percent",
        textinfo='label+percent',
        textposition='outside'
    )])

    title = col.title().split('_')
    text = ' '.join(title[:len(title)//2]) + '<br>' + ' '.join(title[len(title)//2:])
    fig.update_layout(
        title_text=f"{col.title()} Distribution",
        annotations=[dict(
            text=text, 
            x=0.5, 
            y=0.5,
            font_size=20, 
            showarrow=False
        )]
    )

    fig.show()

In [9]:
def eda_proportion(lf: pl.LazyFrame, col: str, n_rows: int = 100, top_n: int = 5) -> None:
    
    count_lf = lf.select(
        pl.col(col).value_counts(sort=True)
    ).unnest(col).with_columns(
        (pl.col('count') / pl.col('count').sum() * 100).round(2).alias('percentage')
    ).sort(by='count', descending=True).head(n_rows)
    
    display(count_lf.collect().to_pandas())
    
    draw_donut_chart(count_lf.collect(), col, top_n)

In [10]:
def overview_col(lf: pl.LazyFrame, col: str, n_rows: int = 100):
    nunique = lf.select(
        pl.col(col).n_unique().alias(f'unique_{col}')
    ).collect().item()
    
    print(f'{col}의 고유 개수: {nunique}')
    
    unique_lf = lf.select(
        pl.col(col).unique().sort().head(n_rows).alias(f'head_{col}'),
        pl.col(col).unique().sort().tail(n_rows).alias(f'tail_{col}'),
    )
    
    display(unique_lf.collect().to_pandas())

In [11]:
def replace_pattern_with_null(lf: pl.LazyFrame, cols: Union[str|List[str]], na_pattern: str):
    """
    지정된 컬럼들에서 정규식 패턴과 매칭되는 값을 null로 변경 
    
    Parameters:
    -----------
    df : pl.DataFrame or pl.LazyFrame
        처리할 데이터프레임
    cols : str or list
        처리할 컬럼명 (단일 컬럼 또는 리스트)
    na_pattern : str
        null로 변경할 정규식 패턴
    
    Returns:
    --------
    pl.DataFrame or pl.LazyFrame
        패턴이 null로 변경된 데이터프레임
    """
    # 단일 컬럼을 리스트로 변환
    if isinstance(cols, str):
        cols = [cols]
    
    replace_null_lf = lf.with_columns(
        pl.when(pl.col(cols).str.to_uppercase().str.contains(na_pattern))
        .then(None)
        .otherwise(pl.col(cols)).name.keep()
    )
    return replace_null_lf

In [12]:
def analyze_null_values(lf: pl.LazyFrame, analysis_cols=None, verbose=True):
    """컬럼별 결측치 분석"""
    
    # 분석할 컬럼 결정
    if analysis_cols is None:
        analysis_cols = lf.collect_schema().names()
    
    # 전체 행 수
    total_rows = lf.select(pl.len()).collect().item()
    
    # 한 번에 null count 계산 후 변환
    null_df = (
        lf.select([pl.col(col).null_count().alias(col) for col in analysis_cols])
        .collect()
        .transpose(include_header=True, header_name='column', column_names=['null_count'])
        .with_columns(
            (pl.col('null_count') / total_rows * 100).round(2).alias('null_pct')
        )
        .sort('null_pct', descending=True)
    )
    
    if verbose:
        print("\n=== 결측치 분석 ===")
        print(f"전체 행 수: {total_rows:,}\n")
        for row in null_df.iter_rows(named=True):
            print(f"{row['column']:45s}: {row['null_count']:>10,}개 ({row['null_pct']:>6.2f}%)")
    
    return null_df

In [13]:
def yn_to_bool(lf: pl.LazyFrame, cols: List[str]) -> pl.LazyFrame:
    """
    Y/N 값을 boolean 타입으로 변환
    
    Parameters:
    -----------
    lf : pl.LazyFrame
        변환할 LazyFrame
    cols : List[str]
        변환할 컬럼 리스트
    
    Returns:
    --------
    pl.LazyFrame
        변환된 LazyFrame
    """
    bool_lf =  lf.with_columns([
        pl.col(col).str.to_uppercase().replace({'Y': True, 'N': False}).alias(col)
        for col in cols
    ])
    return bool_lf

In [14]:
def str_to_categorical(lf: pl.LazyFrame, cols: List[str]) -> pl.LazyFrame:
    # Cast the "text_col" to Categorical
    categorical_lf = lf.with_columns(
        pl.col(cols).cast(pl.Categorical)
    )
    return categorical_lf

In [15]:
# mdr_text 컬럼 처리 

def extract_idx(col, prefix= 'mdr_text_'):
    """
    컬럼 명에서 숫자 추출
    """
    try : 
        return int(col.replace(prefix, '').split('_')[0])
    except : 
        return -1

def get_mdr_cols(df, prefix = 'mdr_text_'):
    """
    mdr_text를 찾아서 역순 정렬

    역순 이유 : mdr_text_39 -> mdr_text_0 순으로 합쳐야 됨
    (최신정보 순으로 보기위함)

    Returns : (type_cols, text_cols)
    """
    cols = df.collect_schema().names()
    # 타입 코드 / 텍스트 컬럼 분리 하는 코드
    type_cols = [col for col in cols if col.startswith(prefix) and col.endswith('_text_type_code')]
    text_cols = [col for col in cols if col.startswith(prefix) and col.endswith('_text')]

    # 숫자 역순 정렬(큰번호부터)
    type_sorted = sorted(type_cols, key = lambda x : extract_idx(x, prefix), reverse = True)
    text_sorted = sorted(text_cols, key = lambda x : extract_idx(x, prefix), reverse = True)

    return type_sorted, text_sorted


def combine_mdr_text(df, prefix='mdr_text_'):
    """ 
    mdr_text 컬럼들 합치기(역순으로)

    상황별 
        1. 둘 다 있을 때
        2. 텍스트만 있을 때
        3. 타입만 있을 때
        4. 둘 다 없을 때
    """
    
    # 컬럼 찾기 (위에서 정의 했던 것)
    type_cols, text_cols = get_mdr_cols(df, prefix)

    # 각 쌍을 "[ReportN: Type] Content" 형식으로
    parts = []
    report_n = 1

    for type_col, text_col in zip(type_cols, text_cols):
        # 인덱스 확인 (같은 번호 끼리 묶기)
        if extract_idx(type_col, prefix) != extract_idx(text_col, prefix):
            continue

        # 둘 다 있을 떼만
        case1 = (
            pl.when(pl.col(type_col).is_not_null() 
            & pl.col(text_col).is_not_null())
            .then(pl.concat_str([
                pl.lit(f'[Report{report_n}: '),
                pl.col(type_col),
                pl.lit(']\n'),
                pl.col(text_col)
            ]))
        )
        
        # 텍스트만 있을 때
        case2 = (
            case1
            .when(pl.col(type_col).is_null()
            & pl.col(text_col).is_not_null())
            .then(pl.col(text_col))
        )

        last_case = case2.otherwise(None)

        parts.append(last_case)
        report_n += 1

# 모든 부분을 /n/n으로 합치기
    if parts:
        combined = pl.concat_str(parts, separator = '\n\n', ignore_nulls = True)
        # 빈 문자열은 null로
        result = pl.when(combined.str.len_chars() > 0).then(combined).otherwise(None)
    else:
        result = pl.lit(None)

    return df.with_columns(result.alias('mdr_text'))        

In [16]:
def safe_groupby_unique(
    lf: pl.LazyFrame, 
    group_cols: List[str],
    agg_cols: List[str] = None,
    top_n: int = 100,
    streaming: bool = True
) -> pl.DataFrame:
    """
    메모리 효율적으로 group by 후 n_unique 계산
    
    Parameters:
    -----------
    lf : pl.LazyFrame
    group_cols : List[str]
        그룹화할 컬럼
    agg_cols : List[str], optional
        unique 개수를 셀 컬럼 (None이면 count만)
    top_n : int
        상위 몇 개만 가져올지
    streaming : bool
        streaming 모드 사용 여부
    """
    if agg_cols is None:
        agg_exprs = [pl.len().alias('count')]
    else:
        agg_exprs = [
            pl.len().alias('count')
        ] + [
            pl.col(col).n_unique().alias(f'{col}_unique')
            for col in agg_cols
        ]
    
    engine = 'streaming' if streaming else 'auto'
    
    return (
        lf.group_by(group_cols)
        .agg(agg_exprs)
        .sort('count', descending=True)
        .head(top_n)
        .collect(engine=engine)
        .to_pandas()
    )

# 사용
result = safe_groupby_unique(
    lf, 
    group_cols=['manufacturer_g1_postal_code'],
    agg_cols=['manufacturer_g1_name'],
    top_n=100
)
result

,manufacturer_g1_postal_code,count,manufacturer_g1_name_unique
0,None,5776432,1940
1,921214386,1104367,3
2,00777-3869,962266,3
3,04002,761576,1
4,69151,406818,1
...,...,...,...
95,2109,8563,2
96,S-431 21,8510,1
97,92612,8448,17
98,803013299,8282,2


## 전처리 이전 전체 Null 세기

In [17]:
overview_col(lf, 'manufacturer_g1_postal_code', 3000)

manufacturer_g1_postal_code의 고유 개수: 3520


,head_manufacturer_g1_postal_code,tail_manufacturer_g1_postal_code
0,None,15086
1,-,151-0073
2,-2113,15130
3,0,1513000
4,0000,15203
...,...,...
2995,95440,Y34 K825
2996,95478,Y34K825
2997,"95478,",Y35D431
2998,95488,ZIP;;


In [18]:
overview_col(lf, 'device_0_udi_di', n_rows=1000)

device_0_udi_di의 고유 개수: 142454


,head_device_0_udi_di,tail_device_0_udi_di
0,None,M2138063M19K15
1,(01)(10)NA,M2138063M215
2,(01)SW11677,M2138063M28K12
3,(01)SW12300,M2138063M4K12
4,(01)UNKNOWN,M2138063M4K15
...,...,...
995,000000643169999800,M986USSL22132600
996,000000643169999817,MHC18020151
997,000000721902425097,MHC18223313
998,000000721902425172,N


In [100]:
device_info_cols = [
    'device_0_manufacturer_d_name', 
    'device_0_brand_name', 
    'device_0_catalog_number', 
    'device_0_model_number', 
    'device_0_udi_public',
    'device_0_device_report_product_code'
]

target =['device_0_udi_di']

my_lf = lf.select(device_info_cols + target)
null_df = analyze_null_values(my_lf, device_info_cols + target)

my_lf.filter(
    True
    # & pl.col(target).is_null()
    # & pl.col(target).is_not_null()
    & pl.col('device_0_manufacturer_d_name').is_not_null()
    & pl.col('device_0_brand_name').is_not_null()
    & pl.col('device_0_catalog_number').is_not_null()
    & pl.col('device_0_model_number').is_not_null()
    # & pl.col('device_0_udi_public').is_null()
).select(pl.col(device_info_cols + target)
# ).select(pl.len()
).sort(device_info_cols
).head(100
).collect().to_pandas()

# +D768RSR48122K0/$$324013143324+ ++M33530010041311$$+730018687/16D20180903 +B004TYYNNNNRM0/$$3211001W52910+

my_lf.group_by(
    pl.col(target)
).agg(
    pl.col('device_0_catalog_number').n_unique().alias('unique_catalog_number')
).sort('unique_catalog_number', descending=True).head(10).collect().to_pandas()

my_lf.filter(
    pl.col(target).eq('00887761000872')
).select(
    pl.col('device_0_catalog_number').value_counts()
).head(10).collect().to_pandas()

my_lf.filter(
    pl.col('device_0_manufacturer_d_name').eq('BAXTER HEALTHCARE CORPORATION')
    & pl.col('device_0_brand_name').eq('VERSACARE MED SURG BED')
    # & pl.col('device_0_catalog_number').eq('P3200H000387')
).head(100).collect().to_pandas()


=== 결측치 분석 ===
전체 행 수: 13,642,636

device_0_udi_di                              :  4,414,173개 ( 32.36%)
device_0_udi_public                          :  4,407,481개 ( 32.31%)
device_0_model_number                        :  3,415,089개 ( 25.03%)
device_0_catalog_number                      :  1,242,658개 (  9.11%)
device_0_manufacturer_d_name                 :     75,939개 (  0.56%)
device_0_brand_name                          :     22,726개 (  0.17%)
device_0_device_report_product_code          :         36개 (  0.00%)


,device_0_manufacturer_d_name,device_0_brand_name,device_0_catalog_number,device_0_model_number,device_0_udi_public,device_0_device_report_product_code,device_0_udi_di
0,BAXTER HEALTHCARE CORPORATION,VERSACARE MED SURG BED,P3200K000035,P3200K,(01)00887761000872(11)160630,FNL,00887761000872
1,BAXTER HEALTHCARE CORPORATION,VERSACARE MED SURG BED,P3200K000004,P3200K,(01)00887761000872(11)111027,FNL,00887761000872
2,BAXTER HEALTHCARE CORPORATION,VERSACARE MED SURG BED,P3200K000425,P3200K,(01)00887761000872(11)141105,FNL,00887761000872
3,BAXTER HEALTHCARE CORPORATION,VERSACARE MED SURG BED,P3200J000524,P3200J,(01)00887761000872(11)110612,FNL,00887761000872
4,BAXTER HEALTHCARE CORPORATION,VERSACARE MED SURG BED,P3200B000019,P3200B,(01)00887761000872(11)041123,FNL,00887761000872
...,...,...,...,...,...,...,...
95,BAXTER HEALTHCARE CORPORATION,VERSACARE MED SURG BED,P3200K000567,P3200K,(01)00887761000872(11)141205,FNL,00887761000872
96,BAXTER HEALTHCARE CORPORATION,VERSACARE MED SURG BED,P3200K000567,P3200K,(01)00887761000872(11)151208,FNL,00887761000872
97,BAXTER HEALTHCARE CORPORATION,VERSACARE MED SURG BED,P3200K000500,P3200K,None,FNL,None
98,BAXTER HEALTHCARE CORPORATION,VERSACARE MED SURG BED,P3200ERENT02,P3200E,None,FNL,None


In [20]:
def safe_groupby_unique(
    lf: pl.LazyFrame, 
    group_cols: List[str],
    agg_cols: List[str] = None,
    top_n: int = 100,
    streaming: bool = True
) -> pl.DataFrame:
    """
    메모리 효율적으로 group by 후 n_unique 계산
    
    Parameters:
    -----------
    lf : pl.LazyFrame
    group_cols : List[str]
        그룹화할 컬럼
    agg_cols : List[str], optional
        unique 개수를 셀 컬럼 (None이면 count만)
    top_n : int
        상위 몇 개만 가져올지
    streaming : bool
        streaming 모드 사용 여부
    """
    if agg_cols is None:
        agg_exprs = [pl.len().alias('count')]
    else:
        agg_exprs = [
            pl.len().alias('count')
        ] + [
            pl.col(col).n_unique().alias(f'unique_{col}')
            for col in agg_cols
        ]
    
    return (
        lf.group_by(group_cols)
        .agg(agg_exprs)
        .sort('count', descending=True)
        .head(top_n)
        .collect(engine="streaming")
        .to_pandas()
    )

# 사용
result = safe_groupby_unique(
    lf, 
    group_cols=['device_0_model_number', 'device_0_brand_name'],
    agg_cols=['device_0_udi_di'],
    top_n=100
)

In [21]:
result

,device_0_model_number,device_0_brand_name,count,unique_device_0_udi_di
0,8100,ALARIS PUMP MODULE,679065,9
1,9445-24,DEXCOM G6 CONTINUOUS GLUCOSE MONITORING SYSTEM,324582,40
2,1002717,T:SLIM X2 INSULIN PUMP WITH CONTROL-IQ TECHNOLOGY,323943,46
3,None,DEXCOM G6 CONTINUOUS GLUCOSE MONITORING SYSTEM,302726,109
4,8100,ALARIS SYSTEM,248479,36
...,...,...,...,...
95,NA,SPECTRUM INFUSION PUMP,15623,8
96,2088TC/52,TENDRIL STS,15438,16
97,None,NOBELREPLACE CC PMC RP 4.3X8MM,15065,6
98,PM2272,ASSURITY MRI,14749,2


In [22]:
col = 'device_0_udi_di'
unique_lf = lf.select(
    pl.col(col).unique().sort().alias(f'{col}'),
)

In [23]:
lf.group_by(['device_0_manufacturer_d_name', 'device_0_model_number', 'device_0_brand_name']).agg(
    pl.col('device_0_udi_di').n_unique().alias('nunique_udi_di')
).sort(by='nunique_udi_di', descending=True).head(1000).collect().to_pandas()

,device_0_manufacturer_d_name,device_0_model_number,device_0_brand_name,nunique_udi_di
0,ENCORE MEDICAL L.P.,None,DJO SURGICAL,639
1,MEDLINE INDUSTRIES LP,None,None,452
2,ALLERGAN (COSTA RICA),None,INSPIRA SMOOTH SILICONE GEL FILLED BREAST IMPLANT,416
3,"MEDLINE INDUSTRIES, LP",None,None,349
4,MENTOR TEXAS,None,MENTOR MEMORYGEL BREAST IMPLANT,333
...,...,...,...,...
995,COOK INC,None,ADVANCE 35 LP LOW PROFILE BALLOON CATHETER,12
996,BECTON DICKINSON MEDICAL SYSTEMS,None,LUER-LOK,12
997,"HEARTWARE, INC.",1650,HEARTWARE VENTRICULAR ASSIST SYSTEM - BATTERY,12
998,AMO PUERTO RICO MFG. INC.,ZMA00,TECNIS IOL,12


## report_to_fda  전처리

In [24]:
col = 'report_to_fda'

eda_proportion(my_lf, col)

,report_to_fda,count,percentage
0,None,7938215,58.19
1,N,2935028,21.51
2,*,1700694,12.47
3,Y,1054990,7.73
4,I,13709,0.10


In [25]:
na_pattern = r'^(\*|I)$'
replace_null_lf = replace_pattern_with_null(my_lf, col, na_pattern)
eda_proportion(replace_null_lf, col, top_n=5)

,report_to_fda,count,percentage
0,None,9652618,70.75
1,N,2935028,21.51
2,Y,1054990,7.73


## event_location 전처리

In [26]:
col = 'event_location'

eda_proportion(my_lf, col)

,event_location,count,percentage
0,None,9346219,68.51
1,I,2541832,18.63
2,OTHER,1147785,8.41
3,OUTPATIENT TREATMENT FACILITY,540836,3.96
4,HOSPITAL,47879,0.35
5,NO INFORMATION,13709,0.10
6,HOME,2950,0.02
7,AMBULATORY SURGICAL FACILITY,917,0.01
8,OUTPATIENT DIAGNOSTIC FACILITY,263,0.00
9,NURSING HOME,242,0.00


In [27]:
na_pattern = r'^(\*|I|N|OTHER|NO INFORMATION|)$'
replace_null_lf = replace_pattern_with_null(lf, col, na_pattern)
eda_proportion(replace_null_lf, col, top_n=3)

,event_location,count,percentage
0,None,13049549,95.65
1,OUTPATIENT TREATMENT FACILITY,540836,3.96
2,HOSPITAL,47879,0.35
3,HOME,2950,0.02
4,AMBULATORY SURGICAL FACILITY,917,0.01
5,OUTPATIENT DIAGNOSTIC FACILITY,263,0.00
6,NURSING HOME,242,0.00


## mdr_report_key 전처리

In [ ]:
col = 'mdr_report_key'

overview_col(my_lf, col)

## manufacturer_link_flag 전처리

In [ ]:
col = 'manufacturer_link_flag'

eda_proportion(my_lf, col)

## device_0_openfda_device_class 전처리

In [ ]:
col = 'device_0_openfda_device_class'

eda_proportion(lf, col, top_n=3)

In [ ]:
na_pattern = r'^(\*|U|N|F|)$'
replace_null_lf = replace_pattern_with_null(lf, col, na_pattern)
eda_proportion(replace_null_lf, col, top_n=3)

## device_0_openfda_device_name 전처리

In [ ]:
col = 'device_0_openfda_device_name'

overview_col(my_lf, col)

In [ ]:
# 방법 2: 공백 제거 포함 (쉼표 앞뒤 공백 처리)
result = (
    lf
    .select(
        pl.col(col)
        .str.split(",")
        .list.eval(pl.element().str.strip_chars())  # 각 요소의 공백 제거
        .alias("device_name")
    )
    .explode("device_name")
    .filter(pl.col("device_name") != "")  # 빈 문자열 제거
    .group_by("device_name")
    .len()
    .sort("len", descending=True)
    .collect(engine='streaming')
)

pprint(sorted(result['device_name']))

## Day of Event로 스코핑

In [ ]:
scope_cols = analysis_cols + [
    'report_number',
    'distributor_name', 'type_of_report', 'date_manufacturer_received', 'source_type', 'date_changed', 'noe_summarized', 
    'device_0_udi_public', 
    'device_0_lot_number'
]

group_cols = [
    'report_number',
    'device_0_manufacturer_d_name',
    'device_0_udi_di', 
    'device_0_udi_public', 
    'device_0_lot_number', 
    'date_of_event',
]

drop_lf = lf.filter(
    pl.col('device_0_udi_public').is_not_null()
    & pl.col('device_0_lot_number').is_not_null()
    & pl.col('report_number').is_not_null()
    & pl.col('device_0_manufacturer_d_name').is_not_null()
    & pl.col('device_0_udi_di').is_not_null()
    & pl.col('date_of_event').is_not_null()
)

analyze_null_values(drop_lf, group_cols)

grouped_lf = drop_lf.select(scope_cols).filter(
    True
    & pl.col('date_of_event').is_not_null()
).group_by(
    group_cols
).agg(pl.len()).sort(by='len', descending=True)

print(f'개수: {grouped_lf.select(pl.len()).collect().item()}')

grouped_df = grouped_lf.head(100).collect().to_pandas()


# duplicate_df[duplicate_df['device_0_udi_di'] == '00763000422615'].transpose()

grouped_df

In [ ]:
drop_lf

In [ ]:
duplicate_df = drop_lf.select(scope_cols).filter(
    True
    & (pl.col('report_number').eq('0001222315-2024-001978'))
    & (pl.col('device_0_manufacturer_d_name').eq('JJGC S.A.'))
    & (pl.col('device_0_udi_di').eq('07899878024170'))
    & (pl.col('device_0_udi_public').eq('07899878024170'))
    & (pl.col('device_0_lot_number').eq('ELWG6'))
    & (pl.col('date_of_event') == '20240614')
).collect().head(10).to_pandas()

duplicate_df.transpose()

## MDR TEXT 스코핑

In [ ]:
scope_cols = analysis_cols + ['distributor_name', 'type_of_report', 'date_manufacturer_received', 'source_type', 'date_changed', 'noe_summarized']

drop_lf = lf.filter(
    pl.col('device_0_udi_di').is_not_null()
)

# analyze_null_values(drop_lf, analysis_cols)

duplicate_df = drop_lf.select(scope_cols).filter(
    pl.col('mdr_text_10_text').is_not_null()
).collect().to_pandas()


duplicate_df['device_0_udi_di'].value_counts()


# duplicate_df[duplicate_df['device_0_udi_di'] == '00763000422615'].transpose()

In [ ]:
# pma_pmn_number
target_df = duplicate_df[duplicate_df['device_0_udi_di'] == '00888707003261'].sort_values('date_received')


target_df['device_date_of_manufacturer'].value_counts()

In [ ]:
target_v2_df = target_df[target_df['device_date_of_manufacturer'] == '20190628']

target_v2_df.transpose()

In [ ]:
target_lf = pl.LazyFrame(target_v2_df)

combine_mdr_text(target_lf).collect().select('mdr_text').to_pandas()

In [ ]:
['Hemolysis', 'Hemorrhage/Blood Loss/Bleeding', 'Heart Failure/Congestive Heart Failure']

In [ ]:
['Anemia', 'Hemorrhage/Blood Loss/Bleeding', 'Muscle Weakness/Atrophy', 'Pleural Effusion', 'Pulmonary Edema', 'Dizziness', 'Numbness', 'Confusion/ Disorientation', 'Syncope/Fainting', 'Melena']